为了节省内存占用。数据预处理的部分如果不是在机器学习中才需要流处理的话，可能就先在清洗后变成csv文件。之后训练的时候再读取了。

In [82]:
# Import Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import networkx as nx
import chardet
import warnings
import pickle
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings('ignore') # 在整个Python脚本执行期间忽略所有的警告。


In [83]:
# Flight_data=pd.read_csv('Flight_Data_TryBig.csv')

# Ticket_Price=pd.read_csv('TicketPrice.csv')



In [84]:
#时间列相除
# Timerols=['DepTime','CRSDepTime','ArrTime','CRSArrTime']

# for col in Timerols:
#     Flight_data[col]=Flight_data[col]//100



# Ticket_Price['Date_of_Journey']=pd.to_datetime(Ticket_Price['Date_of_Journey'],format='%d/%m/%Y')

# Ticket_Price['Dep_Time']=pd.to_datetime(Ticket_Price['Dep_Time'],format='%H:%M')

# Ticket_Price['Month']=Ticket_Price['Date_of_Journey'].dt.month
# Ticket_Price['DayofMonth']=Ticket_Price['Date_of_Journey'].dt.day

# Ticket_Price['DepTime']=Ticket_Price['Dep_Time'].dt.hour

# Ticket_Price.to_csv('TicketPriceTrans.csv',index=False)


In [85]:
#根据之前整理的数据获取票价
# mergedprice = pd.merge(
#     Flight_data,
#     Ticket_Price[['DayofMonth','DepTime','Price']],
#     how='left',
#     left_on=['DayofMonth','DepTime'],
#     right_on=['DayofMonth','DepTime']
# )

# mergedprice.drop_duplicates(inplace=True)

In [86]:
# print(mergedprice.isnull().sum())

In [87]:
# print(mergedprice.info())

In [88]:
# merged_weather_df.to_csv('merged_weatherr1.csv', index=False)

In [89]:
#首先删除数据集中我们不需要处理的列。
# def drop_unnecessary_columns(data,colunms_to_drop):
#     data.drop(columns=columns_to_drop, inplace=True)
#     return data

# columns_to_drop = ['FlightNum']
# drop_unnecessary_columns(mergedprice,columns_to_drop)

# #对cancelcode重新运用None填充规则
# mergedprice['CancellationCode'] = mergedprice['CancellationCode'].fillna('None')

In [90]:
# print(mergedprice.info())

In [91]:
# mergedprice.to_csv('merged_price.csv', index=False)

In [92]:
#首先删除数据集中我们不需要处理的列。
# def drop_unnecessary_columns(data,colunms_to_drop):
#     data.drop(columns=columns_to_drop, inplace=True)
#     return data

# columns_to_drop = ['ArrDelay','DepDelay','Cancelled','CancellationCode','CarrierDelay','WeatherDelay','NASDelay','SecurityDelay','LateAircraftDelay']
# drop_unnecessary_columns(mergedprice,columns_to_drop)

# mergedprice_clean=mergedprice.dropna(subset=['Price'],inplace=True)

# print(mergedprice.isnull().sum())

In [93]:
# print(mergedprice.info())

In [94]:
# mergedprice.to_csv('merged_price_clean.csv', index=False)

In [95]:
# ticketP=mergedprice.drop_duplicates(subset=['TailNum'])

# print(ticketP.info())

# ticketP.to_csv('ticketP.csv', index=False)

In [96]:
ticketModel=pd.read_csv('ticketP.csv')
print(ticketModel.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5155 entries, 0 to 5154
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Month          5155 non-null   int64  
 1   DayofMonth     5155 non-null   int64  
 2   DepTime        5155 non-null   float64
 3   CRSDepTime     5155 non-null   int64  
 4   ArrTime        5155 non-null   float64
 5   CRSArrTime     5155 non-null   int64  
 6   UniqueCarrier  5155 non-null   object 
 7   TailNum        5155 non-null   object 
 8   Origin         5155 non-null   object 
 9   Dest           5155 non-null   object 
 10  Price          5155 non-null   float64
dtypes: float64(3), int64(4), object(4)
memory usage: 443.1+ KB
None


In [97]:
def drop_unnecessary_columns(data,colunms_to_drop):
    data.drop(columns=columns_to_drop, inplace=True)
    return data

columns_to_drop = ['CRSArrTime','CRSDepTime']
drop_unnecessary_columns(ticketModel,columns_to_drop)



,Month,DayofMonth,DepTime,ArrTime,UniqueCarrier,TailNum,Origin,Dest,Price
0,1,9,11.0,13.0,DL,N663DN,LGA,ATL,7202.0
1,1,27,17.0,18.0,DL,N955DL,ATL,JAN,4878.0
2,1,15,13.0,16.0,NW,N509US,EGE,MSP,6961.0
3,1,9,6.0,8.0,DL,N311DL,RIC,CVG,6442.0
4,1,1,9.0,13.0,XE,N23139,TUL,EWR,8907.0
...,...,...,...,...,...,...,...,...,...
5150,12,1,19.0,22.0,XE,N14173,IAH,ORF,10651.0
5151,12,9,11.0,12.0,WN,N486,BWI,CMH,7202.0
5152,12,18,11.0,12.0,EV,N755EV,LEX,ATL,13555.0
5153,12,12,12.0,13.0,WN,N492,HOU,LAX,10991.0


In [98]:
#生成一个新行（未来价格），到时候提交的时候要把注释删掉

# base_row=ticketModel[['Price']]

# min_values=base_row.min()
# max_values=base_row.max()

# random_row = np.random.uniform(min_values, max_values, size=len(base_row))

# 定义一个函数，用于在原始值上随机上下浮动 10%-30%
def random_fluctuation(value):
    # 随机生成一个在 -30% 到 +30% 之间的浮动比例
    fluctuation = np.random.uniform(-0.3, 0.3)
    # 在原始值上进行浮动
    return value * (1 + fluctuation)


ticketModel['FuturePrice'] = ticketModel['Price'].apply(random_fluctuation)

ticketModel['FuturePrice']=ticketModel['FuturePrice'].round(0)

# ticketModel.to_csv('ticketModelView.csv', index=False)

In [99]:
changetormb=['Price','FuturePrice']

for col in changetormb:
    ticketModel[col]=ticketModel[col]/10
    ticketModel[col]=ticketModel[col].round(0)

objcolumns1=ticketModel.select_dtypes(include=['object']).columns

print(objcolumns1)

le=LabelEncoder()

encoders={}

for col in objcolumns1:
    ticketModel[col]=le.fit_transform(ticketModel[col])
    encoders[col] = {
        'encoder': le,
        'mapping': dict(zip(le.classes_, le.transform(le.classes_))),
        'inverse_mapping': {v: k for k, v in dict(zip(le.classes_, le.transform(le.classes_))).items()}
    }
   

print(ticketModel.head())
    

Index(['UniqueCarrier', 'TailNum', 'Origin', 'Dest'], dtype='object')
   Month  DayofMonth  DepTime  ArrTime  UniqueCarrier  TailNum  Origin  Dest  \
0      1           9     11.0     13.0              5     3479     120    12   
1      1          27     17.0     18.0              5     4969      10   107   
2      1          15     13.0     16.0             11     2345      66   149   
3      1           9      6.0      8.0              5      928     179    55   
4      1           1      9.0     13.0             18      648     215    70   

   Price  FuturePrice  
0  720.0        610.0  
1  488.0        430.0  
2  696.0        505.0  
3  644.0        614.0  
4  891.0        702.0  


In [100]:
from sklearn.linear_model import LinearRegression


X=ticketModel.drop(columns=['FuturePrice'])
y=ticketModel['FuturePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



model = LinearRegression()


model.fit(X_train, y_train)


y_pred = model.predict(X_test)


In [101]:
from sklearn.metrics import mean_squared_error, r2_score


mse = mean_squared_error(y_test, y_pred)

n = X_test.shape[0]  # 样本数量
k = X_test.shape[1]  # 特征数量


r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - k - 1)


print(f"Adjusted R^2: {adjusted_r2}")


Mean Squared Error: 28437.277231691693
R^2 Score: 0.8631438174235962
Adjusted R^2: 0.8619374455889365


In [102]:
def encode_category(col, value):
    return encoders[col]['mapping'].get(value, None)

def decode_category(col, encoded_value):
    return encoders[col]['inverse_mapping'].get(encoded_value, None)

In [ ]:
y_test=pd.DataFrame(y_test,columns=['FuturePrice'])
y_pred=pd.DataFrame(y_pred,columns=['FuturePrice'])

print(y_test.info())
print(y_pred.info())

df_result = pd.DataFrame({
    'FormerPrice': y_test['FuturePrice'],
    'FuturePrice': y_pred['FuturePrice']
})

print(df_result.head())

In [105]:
with open('encodersTicket.pkl', 'wb') as f:
    pickle.dump(encoders, f)
    
with open('ticketModel.pkl', 'wb') as f:
    pickle.dump(model, f)